# Document OCR with Surya (Manual)

NOTE: This is a waste of time. Use `modal_surya.py` instead:

- `modal run modal_surya.py::run_surya_extraction --dataset="mydoc"`
- `modal run modal_surya.py::run_surya_extraction --dataset="mychart"`
- `modal run modal_surya.py::run_surya_extraction --dataset="myinfographic"`

In [1]:
import os

import pandas as pd
from PIL import Image
from surya.ocr import run_ocr
from surya.model.detection import segformer
from surya.model.recognition.model import load_model
from surya.model.recognition.processor import load_processor

/home/leloykun/anaconda3/envs/MMFM-Challenge/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
available_gpus

In [3]:
rec_model, rec_processor = load_model(), load_processor()
det_processor, det_model = segformer.load_processor(), segformer.load_model()

/home/leloykun/anaconda3/envs/MMFM-Challenge/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded recognition model vikp/surya_rec on device cuda with dtype torch.float16
Loaded detection model vikp/surya_det2 on device cuda with dtype torch.float16


In [4]:
dataset_name = "mydoc"
dataset_path = os.path.join("data/raw_datasets", dataset_name, "annot_wo_answer.json")
print(dataset_path)
assert os.path.exists(dataset_path)

df_data = pd.read_json(dataset_path)

data/raw_datasets/mydoc/annot_wo_answer.json


In [5]:
BATCH_SIZE = 2
LANGS = ["en", "ar"]

In [ ]:
BATCH_SIZE = 2
LANGS = ["en", "ar"] # Replace with your languages

for i in range(0, len(df_data), BATCH_SIZE):
    batch = df_data[i:i+BATCH_SIZE]
    ids = [
        row["id"]
        for _, row in batch.iterrows()
    ]
    images = [
        Image.open(f"data/raw_datasets/{dataset_name}/images/{row['image']}")
        for _, row in batch.iterrows()
    ]
    print(ids)


    predictions = run_ocr(
        images, [LANGS]*BATCH_SIZE,
        det_model, det_processor, rec_model, rec_processor,
    )
    for prediction in predictions:
        print(prediction)

    texts = [
        " ".join(line.text for line in prediction.text_lines)
        for prediction in predictions
    ]

    for id, text, prediction in zip(ids, texts, predictions):
        with open(f"data/raw_datasets/{dataset_name}/surya/{id}.txt", "w") as f:
            f.write(text)
        with open(f"data/raw_datasets/{dataset_name}/surya/{id}.json", "w") as f:
            f.write(prediction.json())

In [7]:
predictions = run_ocr(
    images, [LANGS]*len(images),
    det_model, det_processor, rec_model, rec_processor,
    batch_size=BATCH_SIZE,
)

Detecting bboxes:   4%|▎         | 1/28 [01:53<51:13, 113.83s/it]

In [ ]:
texts = [
    " ".join(line.text for line in prediction.text_lines)
    for prediction in predictions
]

for id, text, prediction in zip(ids, texts, predictions):
    with open(f"data/raw_datasets/{dataset_name}/surya/{id}.txt", "w") as f:
        f.write(text)
    with open(f"data/raw_datasets/{dataset_name}/surya/{id}.json", "w") as f:
        f.write(prediction.json())